In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from numpy.testing import assert_equal

In [2]:
spark = (SparkSession
         .builder
         .master("local[*]")
         .getOrCreate())

22/03/07 14:44:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/07 14:44:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/07 14:44:28 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/03/07 14:44:28 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/03/07 14:44:28 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


# Problem 1

The text file `/mnt/localdata/public/reddit/comments/RC_2019-02-01-1M` contains the first 1 million comments posted on 2019-02-01 on Reddit. Each line is a JSON object corresponding to a comment.

For this problem, you may **only** use Apache Spark and the Python Standard Library. You **cannot** use numpy, scipy, pandas or scikit-learn. Do **not** print or display large amount of results. You will get deductions if you make the browser unresponsive. Each assertion cell should finish running within 5 minutes.

In [3]:
(spark
 .read.json('/mnt/localdata/public/reddit/comments/RC_2019-02-01-1M')
 .createOrReplaceTempView('comments')) 

22/03/07 14:44:40 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## Problem 1a [15 pts]

Create a function `count_flair_emoji` that returns a SQL statement which will process `comments` to return a Spark DataFrame with two columns: `author` corresponding to a unique `author` and `emoji_flair_count` corresponding to the number of `author_flair_richtext` elements of that `author` that are of `emoji` type. Consider only authors that have more than 2 `author_flair_richtext` elements. Sort by lexicographical order of `author`. 

Hint: `count_flair_emoji` need not only return a string.

In [4]:
from pyspark.sql.functions import udf

@udf
def emoji_count(x):
    
    return len([elem['e'] for elem in x if elem['e']=='emoji'])

spark.udf.register('emoji_count', emoji_count)

<function __main__.emoji_count(x)>

In [5]:
def count_flair_emoji():
    statement = """
    
  
  
    (SELECT DISTINCT(author), emoji_count(author_flair_richtext) AS `emoji_flair_count`
    FROM comments 
    WHERE author IS NOT NULL
        AND author_flair_richtext IS NOT NULL 
        AND SIZE(author_flair_richtext) != 0
        AND SIZE(author_flair_richtext) > 2
    
    ORDER BY author
    )
   
    
    """
    return statement

# WHERE author == '-Jack_MeHoff-' or author == '-NoFaithInFate-' or author == '-weirdo-'
# author_flair_richtext

In [6]:
df_emoji = spark.sql(count_flair_emoji()).limit(10).toPandas()
assert_equal(
    df_emoji.iloc[:5].to_numpy().tolist(),
    [['-Jack_MeHoff-', '3'],
     ['-NoFaithInFate-', '3'],
     ['-Reddit_Account-', '3'],
     ['-badger--', '2'],
     ['-weirdo-', '7']]
)

## Problem 1b [10 pts]

Create a function `count_comments` that returns a SQL statement which will process `comments` to return a Spark DataFrame with three columns: `author` corresponding to a unique `author`, `comment_count` corresponding to the number of comments by the `author` and `subreddit_count` corresponding to the number of subreddits the `author` posted a comment on. Exclude `[deleted]` authors. Sort by decreasing `comment_count`.

In [7]:
def count_comments():
    statement = """
    SELECT *
    FROM
    (SELECT author, count(*) as `comment_count`, COUNT(DISTINCT(subreddit)) as `subreddit_count`
    FROM comments
    
    GROUP BY author)
    
    WHERE author != '[deleted]'
    ORDER BY comment_count DESC
    
    
    """
    return spark.sql(statement)

In [8]:
df_count = count_comments().limit(10).toPandas()
assert_equal(
    df_count.iloc[:10].to_numpy().tolist(),
    [['AutoModerator', 12307, 1461],
     ['transcribot', 621, 1],
     ['MemeInvestor_bot', 593, 1],
     ['KeepingDankMemesDank', 478, 1],
     ['transcribersofreddit', 422, 1],
     ['Marketron-I', 329, 1],
     ['RickyontheRadiator', 318, 3],
     ['NFCAAOfficialRefBot', 282, 1],
     ['agree-with-you', 279, 240],
     ['imguralbumbot', 223, 186]]
)